In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import numpy as np

import torch     
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

In [5]:
SOS_token = 0
EOS_token = 1


class ShapeChecker():
    def __init__(self, name):
        self.name = name
        self.shapes = {} # Keep a cache of every axis-name seen

    def __call__(self, tensor, names, broadcast=False):
        parsed = einops.parse_shape(tensor, names)

        for name, new_dim in parsed.items():
            old_dim = self.shapes.get(name, None)

            if (broadcast and new_dim == 1):
                continue
            
            if old_dim is None:
                self.shapes[name] = new_dim
                continue
        
            if new_dim != old_dim:
                raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")
        

In [14]:
def loadData(path1, path2, reverse=False):
    # input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    # print("Read %s sentence pairs" % len(pairs))
    lines1 = np.array(open(path1, encoding='utf-8').\
         read().strip().split('\n'))
    lines2 = np.array(open(path2, encoding='utf-8').\
         read().strip().split('\n'))

    return lines1, lines2


In [15]:
target_raw, context_raw = loadData("informal.txt", "formal.txt")

In [16]:
print(context_raw[-1])
print(target_raw[-1])

I do have it. Please send me a message and I will send it to you.
I have it if you would like it... message me and I'll gladly send it to you!!


In [17]:
BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 32

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8


train_raw = (
    torch.utils.data.ConcatDataset(context_raw[is_train], target_raw[is_train]))


TypeError: ConcatDataset.__init__() got an unexpected keyword argument 'shuffle'